# Maths Lab - 4

## Teammates:
#### Akshita Potdar - 702768460
#### Kalyani Khandait - 702768391
#### Logesh Gangadharan - 702723890


##  Problem -1

Use the following steps to convince yourself that the theorem holds:
1 Generate X, y at random.
2 Solve the lasso problem with λ = λmax.
3 Compute M = max |β∗
i |.
Repeat the above experiment N times for a large value of N . If Mi
is the value of M obtained during the i-th experiment, compute
max
i=1,...,N Mi
and make sure M = 0 (up to numerical precision).

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Generating random data for linear regression
np.random.seed(42)
x = 2 * np.random.rand(100, 1)
y = 4 + 3 * x + np.random.randn(100, 1)

n=40
p=10
N=100

coef=[]

for _ in range(N):
  lambda_max = (1/n) * np.max(np.abs(x.T @y))
  lasso = Lasso(alpha = lambda_max, fit_intercept= False)
  lasso.fit(x,y)

  # for the formula M= max of Beta star
  beta_star = lasso.coef_

  M = np.max(np.abs(beta_star))
  coef.append(M)


M_max = np.max(coef)
print(f'Max M across {N} experiments: {M_max:.10f}')


Max M across 100 experiments: 0.0000000000


## Problem - 2

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def coordinate_descent(A, y, tol=1e-6, positive=False, max_iter=1000):
    m, n = A.shape
    x = np.zeros(n)
    residuals = y - A.dot(x)
    diff = np.inf
    iter_count = 0

    while diff > tol and iter_count < max_iter:
        x_old = x.copy()
        for j in range(n):
            # Calculate the partial residual without the contribution of x_j
            residual_no_j = residuals + A[:, j] * x[j]
            # Update x_j based on the partial residual and constraint
            rho = np.dot(A[:, j], residual_no_j)
            if positive:
                x[j] = max(0, rho / np.dot(A[:, j], A[:, j]))
            else:
                x[j] = rho / np.dot(A[:, j], A[:, j])
            # Update residuals
            residuals = residual_no_j - A[:, j] * x[j]

        diff = np.linalg.norm(x - x_old)
        iter_count += 1

    return x


np.random.seed(0)
A = np.random.randn(100, 10)
y = A.dot(np.random.randn(10)) + np.random.randn(100)

# Coordinate descent solution with positive coefficients
x_cd_positive = coordinate_descent(A, y, positive=True)

# Linear Regression for comparison
lr = LinearRegression(fit_intercept=False)
lr.fit(A, y)
x_lr = lr.coef_
x_lr_positive = np.maximum(x_lr, 0)

# Print results
print("Coordinate Descent Positive Coefficients:", x_cd_positive)
print("Linear Regression Coefficients", x_lr_positive)
print("Difference between Coordinate Descent Positive and Adjusted Linear Regression:", np.linalg.norm(x_cd_positive - x_lr_positive))


Coordinate Descent Positive Coefficients: [0.44908691 0.81784225 0.         0.25489193 0.4126498  0.31741838
 0.62476308 0.         0.         0.21062992]
Linear Regression Coefficients [0.60384959 0.83015874 0.         0.12480557 0.28558375 0.26861244
 0.53156047 0.         0.         0.28404853]
Difference between Coordinate Descent Positive and Adjusted Linear Regression: 0.2713493376607383


## Problem - 3

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso

#function for soft thresholding
def soft_thresholding(rho, lambda_):
    if rho < -lambda_:
        return (rho + lambda_)
    elif rho > lambda_:
        return (rho - lambda_)
    else:
        return 0


#fucntion for coordinate-descent algorithm for lasso regression
def lasso_coordinate_descent(X, y, lambda_, max_iter=1000, tol=1e-6):
    n, p = X.shape
    beta = np.zeros(p)

    for iteration in range(max_iter):
        beta_old = np.copy(beta)

        for j in range(p):

            residual = y - (X @ beta) + X[:, j] * beta[j]

            # Calculating rho
            rho = np.dot(X[:, j], residual)


            beta[j] = soft_thresholding(rho, lambda_) / np.sum(X[:, j]**2)


        if np.sum(np.abs(beta - beta_old)) < tol:
            break

    return beta

# Generating random data
np.random.seed(0)
n, p = 50, 10  # 50 samples and 10 features
X = np.random.randn(n, p)  # Random feature matrix
y = np.random.randn(n)     # Random response vector


alpha = 0.1  # Regularization parameter
lambda_ = alpha * n
beta_custom = lasso_coordinate_descent(X, y, lambda_)

# Sklearn Lasso implementation
lasso = Lasso(alpha=alpha, fit_intercept=False, max_iter=1000, tol=1e-6)
lasso.fit(X, y)
beta_sklearn = lasso.coef_

#putput the coefficients
print("Custom implementation coefficients (larger dataset):\n", beta_custom)
print("Scikit-learn Lasso coefficients (larger dataset):\n", beta_sklearn)

#comparing the difference between the coefficients
difference = np.linalg.norm(beta_custom - beta_sklearn)
print(f"Difference between custom and sklearn coefficients: {difference:.10f}")



Custom implementation coefficients (larger dataset):
 [ 0.1208606   0.          0.0763886   0.         -0.10401496 -0.00296079
  0.1302818   0.05173474  0.          0.04710325]
Scikit-learn Lasso coefficients (larger dataset):
 [ 0.12086055 -0.          0.07638862  0.         -0.10401498 -0.00296079
  0.13028181  0.05173474 -0.          0.04710325]
Difference between custom and sklearn coefficients: 0.0000000580
